PDF dosyasını yüklemek için PyPDFLoader kullanıyoruz

In [25]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "QUANTUM COMPUTING.pdf"
loader = PyPDFLoader(file_path)

In [26]:
data = loader.load()
data[0]

Document(metadata={'producer': 'Aspose.Pdf.Kit for Java 4.3.0', 'creator': 'Acrobat PDFMaker 23 for Word', 'creationdate': '2023-07-31T13:50:09-04:00', 'author': 'Boyle, Kathleen [ICG-CGI]', 'basewrap': 'CIRB1130', 'ciradocversion': '2016', 'company': 'CITI', 'documentlcid': '1033', 'favesessionid': '41341115-d617-4955-b016-b8d6122842eb', 'gps': 'GPS', 'gpstext': 'GLOBAL POTENTIAL SOURCES', 'keywords': 'URL1=https://www.citivelocity.com/rendition/authfilelinksvcs/eppublic/V1/file?paramData=ZmlsZU5hbWU9L2dwcy9yc2NoX3BkZl8zMDE3MTA5Mi5wZGYmcGxhdGZvcm1JZD00Ng&&URL2=https://www.citivelocity.com/rendition/authfilelinksvcs/eppublic/V1/file?paramData=ZmlsZU5hbWU9L2dwcy9yc2NoX3BkZl8zMDE3MTA5Mi5wZGYmcGxhdGZvcm1JZD00Ng', 'msip_label_dd181445-6ec4-4473-9810-00785f082df0_actionid': '444c54a3-acea-481a-a036-d2b0cfddda97', 'msip_label_dd181445-6ec4-4473-9810-00785f082df0_contentbits': '0', 'msip_label_dd181445-6ec4-4473-9810-00785f082df0_enabled': 'true', 'msip_label_dd181445-6ec4-4473-9810-00785f082

In [27]:
len(data)

152

 Metinleri parçalara ayırmak için RecursiveCharacterTextSplitter kullanacağız

- Dil modelleri uzun metinleri okuyamaz, bu yüzden metni küçük parçalara (chunk'lara) ayırırız.
- Böylece soru-cevaplar daha doğru, arama daha etkili olur.
- Kısa parçalar, embedding kalitesini de artırır.

In [28]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter( chunk_size=1000)
docs = text_splitter.split_documents(data)

print(f"Total number of chunks: {len(docs)}")

Total number of chunks: 473


In [29]:
docs[10]

Document(metadata={'producer': 'Aspose.Pdf.Kit for Java 4.3.0', 'creator': 'Acrobat PDFMaker 23 for Word', 'creationdate': '2023-07-31T13:50:09-04:00', 'author': 'Boyle, Kathleen [ICG-CGI]', 'basewrap': 'CIRB1130', 'ciradocversion': '2016', 'company': 'CITI', 'documentlcid': '1033', 'favesessionid': '41341115-d617-4955-b016-b8d6122842eb', 'gps': 'GPS', 'gpstext': 'GLOBAL POTENTIAL SOURCES', 'keywords': 'URL1=https://www.citivelocity.com/rendition/authfilelinksvcs/eppublic/V1/file?paramData=ZmlsZU5hbWU9L2dwcy9yc2NoX3BkZl8zMDE3MTA5Mi5wZGYmcGxhdGZvcm1JZD00Ng&&URL2=https://www.citivelocity.com/rendition/authfilelinksvcs/eppublic/V1/file?paramData=ZmlsZU5hbWU9L2dwcy9yc2NoX3BkZl8zMDE3MTA5Mi5wZGYmcGxhdGZvcm1JZD00Ng', 'msip_label_dd181445-6ec4-4473-9810-00785f082df0_actionid': '444c54a3-acea-481a-a036-d2b0cfddda97', 'msip_label_dd181445-6ec4-4473-9810-00785f082df0_contentbits': '0', 'msip_label_dd181445-6ec4-4473-9810-00785f082df0_enabled': 'true', 'msip_label_dd181445-6ec4-4473-9810-00785f082

In [30]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

True

GoogleGenerativeAIEmbeddings: Google’ın AI modelini kullanarak bir metni vektöre dönüştürür

In [31]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello, world!")
vector[:5]

[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734263241291,
 0.01813093200325966]

In [32]:
vector_store=Chroma.from_documents(embedding=embeddings, documents=docs)

In [33]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 10})

In [34]:
retrieved_docs = retriever.invoke("What is quantum computing?")
print(f"Total number of chunks: {len(retrieved_docs)}")
print(retrieved_docs[5].page_content)

Total number of chunks: 10
2022 2023 2024 2025 2026 2027
($bn)
Range of Current TAM Forecasts (2022-27E)  
Source: Industry forecasts
10
9
8
7
6
5
4
3
2
1
0
2.0
1.5
1.0
0.5
0.0
2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022
Angel & Seed Early-stage VC Later-stage VC
($bn)
VC Investment in Quantum Computing (2010-22)  
Source: PitchBook Data Inc.
120
100
80
60
40
20
0
2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022
($bn)
VC Investment in Quantum Computing vs. Artificial Intelligence  
Source: PitchBook Data Inc.
Quantum Computers Artificial Intelligence
THE QUANTUM COMPUTING OPPORTUNITY
WHY QUANTUM COMPUTING IS A BIG DEAL
Computing is at an inflection point. After five 
decades of increasing computational power  
by Moore’s Law (i.e., doubling the number of 
transistors per chip every 2 years), the growth  
rate of classical computing is reaching its physical 
limit. At the same time, the capabilities of quantum 
computers (QCs) are advancing at a “dou

Google Gemini API kullanarak LLM invoke

In [35]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.3,
    max_tokens=512,

)

In [39]:
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

In [ ]:
system_prompt = (
    "You are a helpful assistant that provides information about quantum computing. "
    "You will be provided with some documents and you will answer the user's question based on those documents. "
    "If you don't know the answer, say 'I don't know'. "
    "If the answer is not in the documents, say 'I don't know'. "
    "If the answer is in the documents, provide a detailed answer. "
    "Be concise and clear. "
    "You can use the information from the documents.\n\n"
    "The documents are: {context}"
)

In [38]:
prompt=ChatPromptTemplate.from_messages(
    [
    ("system",system_prompt),
    ("human","{input}")
    ]
)

LLM+PROMT

In [40]:
chain = create_stuff_documents_chain(llm, prompt)

RAG+LLM

In [41]:
retrieval_chain = create_retrieval_chain(retriever, chain)

In [47]:
response = retrieval_chain.invoke({"input":"What is quantum computing?"})
print(response["answer"])

Quantum computing is the next phase of computing, moving from theoretical to practical. It harnesses the principles of quantum mechanics to perform tasks faster, more efficiently, and cheaper than classical computers.  Unlike classical computers that store information as bits representing 0 or 1, quantum computers use qubits. Qubits, through superposition, can be in a combination of both states (0 and 1) simultaneously.  Furthermore, quantum entanglement allows every additional qubit to double the potential states of a quantum computer. This allows quantum computers to tackle complex problems beyond the capabilities of classical computers.
